In [1]:
# import psycopg2
from sqlalchemy import create_engine
import pandas as pd

# Create a class to create pandas dataframe from SQL queries

In [24]:
#Read data from JDBC server
class DB:
    def __init__(self):
        self.tables = None
        self.df = None
        self.point = None
        self.POSTGRES_HOST = "codd03.research.northwestern.edu"
        self.POSTGRES_PORT = 5432
        self.POSTGRES_DBNAME = 'cpdb'
        self.POSTGRES_USERNAME = 'cpdb_student'
        self.POSTGRES_PASSWORD = 'DataSci4Lyfe'
        self.postgres_str = ('postgresql://{username}:{password}@{host}:{port}/{dbname}'.format(username=self.POSTGRES_USERNAME,
                                                                                           password=self.POSTGRES_PASSWORD,
                                                                                           host=self.POSTGRES_HOST,
                                                                                           port=self.POSTGRES_PORT,
                                                                                           dbname=self.POSTGRES_DBNAME))
        self.engine = create_engine(self.postgres_str)

    def query(self, queryCommand):
            self.point = self.engine.execute(queryCommand)
            self.df = None
            try: self.df = pd.DataFrame([*self.point.mappings()])
            except Exception as msg:
                if msg == 'This result object does not return rows. It has been closed automatically.':pass
            return self.df

    def get_tables_from_commands(self, commands=None):
        self.tables = {}
        for command in commands:
            try:
                df = self.query(command)
                if df is not None:
                    self.tables[f't{len(self.tables.keys())}'] = {'frame': df, 'sql': command}
            except:
                pass
        return self.tables



In [25]:
# test it out

def test():
    obj_test = DB()
    t = obj_test.get_tables_from_commands

    query1 = '''
    DROP TABLE if exists area_ids;
    CREATE TEMP TABLE area_ids AS
        ( SELECT id, name FROM data_area
          WHERE area_type='police-districts');
    SELECT * FROM area_ids;

    -- Determining civilian population of each PD
    DROP TABLE IF EXISTS popul_by_pd;
    CREATE TEMP TABLE popul_by_pd AS
        ( SELECT area_id, SUM("count") population FROM data_racepopulation
          WHERE area_id IN (SELECT id FROM area_ids)
          group by area_id);
    select * from popul_by_pd;
    '''

    commands = query1.split(';')
    tables = t(commands)
    if len(tables.keys()) == 2: print("Passed!")
    else: print("An error occurred")

    del obj_test

test()

Passed!


# Get all required tables

In [26]:
def table_from_file(filename):
    obj = DB()
    t = obj.get_tables_from_commands
    with open(filename, 'r') as f:
        sqlFile = f.read()
    commands_raw = sqlFile.split(';')
    commands = []
    for command in commands_raw:
        q = f'''
        {command}
        '''
        commands.append(q)
    tables = t(commands)
    total = len(tables.keys())
    print(total, "tables received")
    del obj
    return (total, tables)

### console1.sql

In [27]:
filename = 'console1.sql'
total, tables = table_from_file(filename)

2 tables received


In [29]:
df0 = tables['t0']['frame']
q0 = tables['t0']['sql']
print(q0)
df0.head()



        

select * from data_area_beat
        


,name,side
0,3100,north
1,1655,north
2,1613,north
3,1611,north
4,1621,north


In [30]:
df1 = tables['t1']['frame']
q1 = tables['t1']['sql']
print(q1)
df1.head()



        
select * from trr_trr
         where id is not null
           and beat is not null
        


,beat,block,created_at,crid,direction,event_id,firearm_used,id,indoor_or_outdoor,lighting_condition,...,subject_gender,subject_id,subject_injured,subject_race,taser,total_number_of_shots,trr_created,trr_datetime,updated_at,weather_condition
0,513,110XX,2021-11-22 21:11:10.744429+00:00,None,S,None,None,428753,Outdoor,DAYLIGHT,...,M,NaN,None,BLACK,None,NaN,2018-01-25 04:38:00,2018-01-25 21:11:00+00:00,2021-11-22 21:11:10.744429+00:00,CLEAR
1,725,14XX,2021-11-22 21:11:10.744429+00:00,None,W,None,None,4042119,Indoor,ARTIFICIAL,...,F,NaN,None,BLACK,None,NaN,2019-11-05 07:30:00,2019-11-05 22:42:00+00:00,2021-11-22 21:11:10.744429+00:00,CLOUD
2,2533,17XX,2021-11-22 21:11:10.744429+00:00,None,N,None,None,700207,Outdoor,ARTIFICIAL,...,M,NaN,None,BLACK,None,NaN,2018-11-11 02:03:00,2018-11-11 04:00:00+00:00,2021-11-22 21:11:10.744429+00:00,CLEAR
3,2533,17XX,2021-11-22 21:11:10.744429+00:00,None,N,None,None,700194,Outdoor,ARTIFICIAL,...,M,NaN,None,BLACK,None,NaN,2018-11-11 01:41:00,2018-11-11 04:00:00+00:00,2021-11-22 21:11:10.744429+00:00,CLEAR
4,1224,17XX,2021-11-22 21:11:10.744429+00:00,None,W,None,None,388829,Outdoor,DAYLIGHT,...,M,NaN,None,BLACK,None,NaN,2017-12-01 04:49:00,2017-12-01 19:50:00+00:00,2021-11-22 21:11:10.744429+00:00,CLEAR


In [38]:
missing = df1.isnull().sum()

pandas.core.series.Series

In [ ]:
# drop columns with 75% data missing

